# rPPG Predictions

This notebook runs various pretrained models from [rPPG-Toolbox](https://github.com/ubicomplab/rPPG-Toolbox) to extract rPPG signals from videos. The rPPG signals are then saved to disk for further analysis.

In [ ]:
import numpy as np
import torch
import respiration.utils as utils

dim = 72
device = utils.get_torch_device()

# Models are trained on different datasets
models = {
    'TS-CAN': [
        'BP4D_PseudoLabel_TSCAN',
        'MA-UBFC_tscan',
        'PURE_TSCAN',
        'SCAMPS_TSCAN',
        'UBFC-rPPG_TSCAN',
    ],
    'DeepPhys': [
        'BP4D_PseudoLabel_DeepPhys',
        'MA-UBFC_deepphys',
        'PURE_DeepPhys',
        'SCAMPS_DeepPhys',
        'UBFC-rPPG_DeepPhys',
    ],
}

In [ ]:
from respiration.extractor.ts_can import TSCAN
from respiration.extractor.deep_phys import DeepPhys


def load_model(name, path) -> torch.nn.Module:
    """
    Load a pretrained model from the rPPG-Toolbox.
    """
    match name:
        case 'DeepPhys':
            loaded_model = DeepPhys(img_size=dim).to(device)
        case 'TS-CAN':
            loaded_model = TSCAN(img_size=dim).to(device)
        case _:
            raise ValueError(f'Unknown model: {name}')

    loaded_model = torch.nn.DataParallel(loaded_model).to(device)
    loaded_model.load_state_dict(torch.load(path, map_location=device))
    return loaded_model


def frames_preprocessing(name: str, frames: np.ndarray):
    raw, diff = utils.preprocess_video_frames(frames, dim)

    # Permute from (T, H, W, C) to (T, C, H, W)
    raw = torch.tensor(raw).permute(0, 3, 1, 2).to(device)
    diff = torch.tensor(diff).permute(0, 3, 1, 2).to(device)

    # Concatenate the two inputs
    combined = torch.cat((diff, raw), dim=1).to(device)

    # TS-CAN requires the input to be a multiple of frame-depth (20)
    if name == 'TS-CAN':
        cut_off = (raw.shape[0] // 20) * 20
        combined = combined[:cut_off]

    return combined

In [ ]:
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()
scenarios = dataset.get_scenarios(['101_natural_lighting'])

In [ ]:
from tqdm.auto import tqdm

predictions = []

for (subject, setting) in tqdm(scenarios):
    print(f'Processing: {subject} {setting}')
    frames, meta = dataset.get_video_rgb(subject, setting)

    for model_name, model_version in models.items():
        for model in model_version:
            print(f'--> {model_name} {model}')
            model_path = utils.file_path('data', 'rPPG-Toolbox', model + '.pth')
            model = load_model(model_name, model_path)

            input_frames = frames_preprocessing(model_name, frames)

            with torch.no_grad():
                prediction = model(input_frames)

            prediction = prediction.cpu().numpy().squeeze()

            predictions.append({
                'model_name': model_name,
                'model': model_version,
                'subject': subject,
                'setting': setting,
                'sampling_rate': meta.fps,
                'signal': prediction.tolist(),
            })

In [ ]:
import pandas as pd

df = pd.DataFrame(predictions)
df.head()

In [ ]:
signals_dir = utils.dir_path('outputs', 'signals', mkdir=True)
signals_path = utils.join_paths(signals_dir, 'r_ppg_predictions.csv')
df.to_csv(signals_path, index=False)